In [1]:
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import (PartGroup, Part, show as show_object)

set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)
import cadquery as cq


Overwriting auto display for cadquery Workplane and Shape


/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [7]:
BAT_MOUNT_DIM = [99, 70, 20]
BAT_MOUNT_FILLET = [15, 10]
BAT_MOUNT_HOLES = [(-39,0), (29.5, -24.5), (29.5, 24.5)]
POST_HOLE_DIA = 2.5
ARM_DIM = [70,20]
ARM_HOLE_DIA = 5
WIRE_HOLE_DIA = 4.2
THICK = 5
NARMS = 3
SPINCASTER_DIA = 18.5*25.4

caster = (
    cq.Workplane("YZ").circle(SPINCASTER_DIA/2).extrude(BAT_MOUNT_DIM[0], both=True)
    .translate((0,0,-SPINCASTER_DIA/2+THICK/2))
)
arm_intersect = (
    cq.Workplane("YZ").circle(SPINCASTER_DIA/2).circle(SPINCASTER_DIA/2+THICK)
    .extrude(BAT_MOUNT_DIM[0], both=True)
    .translate((0,0,-SPINCASTER_DIA/2+THICK/2))
)
arm_intersect

arm = (
    cq.Workplane().box(*ARM_DIM, 100, centered=[False, True, True])
    .edges("|Z and >X").fillet(ARM_DIM[1]/2 - 0.001)
    .faces(">Z").workplane()
    .move(ARM_DIM[0]-ARM_DIM[1]/2, 0)
    .cboreHole(ARM_HOLE_DIA, 1.5*ARM_HOLE_DIA, 4)
)
base = cq.Workplane()
for i in [0, 80, -80]:
    base = base.union(arm.rotate((0,0,0),(0,0,1), i))
base = base.intersect(arm_intersect)
base

inside = (
    cq.Workplane().box(BAT_MOUNT_DIM[0]-2*THICK, BAT_MOUNT_DIM[1]-2*THICK, BAT_MOUNT_DIM[2], centered=[True, True, False])
    .translate((0,0,THICK))
    .edges("<X and |Z").fillet(BAT_MOUNT_FILLET[0])
    .edges(">X and |Z").fillet(BAT_MOUNT_FILLET[1])
    .union(
        cq.Workplane("XZ").center(-5,0).slot2D(2*WIRE_HOLE_DIA, WIRE_HOLE_DIA).extrude(100, both=True)
        .translate((0,0,2*THICK+WIRE_HOLE_DIA/2))
    )
)
    
part = (
    cq.Workplane().box(*BAT_MOUNT_DIM, centered=[True, True, False])
    .edges("<X and |Z").fillet(BAT_MOUNT_FILLET[0])
    .edges(">X and |Z").fillet(BAT_MOUNT_FILLET[1])
    .union(base)
    .cut(inside)
    .faces(">Z").workplane().pushPoints(BAT_MOUNT_HOLES)
        .circle(POST_HOLE_DIA/2)
        .rect(POST_HOLE_DIA*6, POST_HOLE_DIA*6)
        .extrude(-BAT_MOUNT_DIM[2])
    .cut(caster)
)


part = part.rotate((0,0,0), (0,1,0),180).rotate((0,0,0),(0,0,1), -90)
part

Done, using side car 'Cadquery'


# Spincaster Battery Mount

This part connects the top half of https://www.thingiverse.com/thing:4497887 to the side of the spincaster, with M5 machine screws.

In [ ]:
show_object(part)

In [8]:
from pathlib import Path; path= str(Path.home() / "Downloads") + "/spincaster_battery_base.stl"
from mkr3d.commands import post
cq.exporters.export(part, path)
post(path, slice=True, upload=True, queue=False, publish=False)
print("done")

{'exec_dir': '/home/semartin/Documents/devenv/mkr3d/grid-apps'}
/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/mkr3d/kirimoto.py
node /home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/mkr3d/kiri-cli.js /home/semartin/Documents/devenv/mkr3d/grid-apps /home/semartin/.config/mkr3d/km_slicer.json /home/semartin/.config/mkr3d/km_device.json /home/semartin/Downloads/spincaster_battery_base.stl /home/semartin/Downloads/spincaster_battery_base.gcode
settings_path /home/semartin/.config/mkr3d/km_slicer.json
device_path /home/semartin/.config/mkr3d/km_device.json
{
  msg: 'failed to load',
  dir: '/home/semartin/Documents/devenv/mkr3d/grid-apps',
  file: '/home/semartin/Documents/devenv/mkr3d/grid-apps/src/geo/debug.js'
}
{ fake_worker: '/code/worker.js?3.0.5' }
kiri | init work | 3.0.5
[
  'driver',     'loader',    'license',
  'version',    'copyright', 'conf',
  'work',       'client',    'newEngine',
  'api',        'Top',       'Slice',
  'newTop',     'new